# SiRF Scanner

## Written by Michael George

Iterate through folders, looking for file reader issues caused by wrong extension, missing headers, bad checksums, etc.

In [1]:
import os
import sys

import pprint
import traceback

import numpy as np

readersPath = os.path.join('.', 'readers')
if readersPath not in sys.path:
    sys.path.extend([readersPath])

from generic_reader import getReader

## Main Function

In [2]:
def scanFiles():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions')

    exceptions = {}
    sats = {}
    satMeans = {}
    
    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getReader(filePath)
                try:
                    reader.load()

                    # Record satellites
                    if reader.device in sats:
                        sats[reader.device].append(reader.data['sat'])
                    else:
                        sats[reader.device] = [reader.data['sat']]

                    # Record mean
                    satMean = reader.data['sat'].mean()
                    if reader.device in satMeans:
                        satMeans[reader.device].append(satMean)
                    else:
                        satMeans[reader.device] = [satMean]

                    print('.', end='')

                except Exception:
                    exceptions[file] = traceback.format_exc()

                    print('E', end='')
                
    # Show stats
    print(os.linesep * 2 + 'Satellite Means:')
    pp = pprint.PrettyPrinter(indent=4)
    for device in sats:
        print(device)
        tmp = np.concatenate(sats[device], axis=0)
        unique, counts = np.unique(tmp, return_counts=True)
        print(np.asarray((unique, counts)).T)

    # Report errors
    print(os.linesep * 2 + 'Errors:')
    if len(exceptions) > 0:
        for file in exceptions:
            print(os.linesep + file)
            print(exceptions[file].split(os.linesep)[-2:-1])
    else:
        print(os.linesep + 'None!')

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

    scanFiles()    

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................